# **ANN churn dataset**

In [29]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from keras.layers import Dropout


data = pd.read_csv('Churn_Modelling.csv')

X = data.iloc[:,3:13].values
Y = data.iloc[:,13].values

labelEncoder_gen = LabelEncoder()
X[:,2] = labelEncoder_gen.fit_transform(X[:,2])

ct = ColumnTransformer([('oh_enc',OneHotEncoder(sparse=False),[1],)],remainder='passthrough')
X = ct.fit_transform(X)
X = X[:,1:]
sc = StandardScaler()
X = sc.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)


def build_classifier(optimizer):
  classif = Sequential()
  classif.add(Dense(activation = 'relu', input_dim=11,units = 22, kernel_initializer = 'uniform'))
  classif.add(Dropout(rate=0.1))
  classif.add(Dense( activation = 'relu', units = 22, kernel_initializer = 'uniform'))
  classif.add(Dropout(rate=0.1))
  classif.add(Dense( activation = 'sigmoid', units = 1, kernel_initializer = 'uniform'))
  classif.compile(optimizer=optimizer,loss = 'binary_crossentropy',metrics = ['accuracy'])
  return classif

classifier = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size':[25,32],'epochs':[50,70],'optimizer':['adam','rmsprop']}
grids = GridSearchCV(estimator=classifier,param_grid=parameters,scoring='accuracy',cv=3)
grids.fit(X_train,Y_train)
best_parameters = grids.best_params_
best_accuracy = grids.best_score_

print(best_parameters)
print(best_accuracy)

Epoch 1/50
214/214 [==============================] - 1s 1ms/step - loss: 0.5953 - accuracy: 0.8024
Epoch 2/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4278 - accuracy: 0.8024
Epoch 3/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4341 - accuracy: 0.7960
Epoch 4/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4115 - accuracy: 0.8106
Epoch 5/50
214/214 [==============================] - 0s 2ms/step - loss: 0.4089 - accuracy: 0.8289
Epoch 6/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4085 - accuracy: 0.8296
Epoch 7/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4178 - accuracy: 0.8173
Epoch 8/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4024 - accuracy: 0.8443
Epoch 9/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4028 - accuracy: 0.8384
Epoch 10/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4105 - accuracy: 0.8272

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Epoch 1/50
214/214 [==============================] - 1s 1ms/step - loss: 0.6121 - accuracy: 0.7751
Epoch 2/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4319 - accuracy: 0.7926
Epoch 3/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.7981
Epoch 4/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4133 - accuracy: 0.8252
Epoch 5/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4263 - accuracy: 0.8294
Epoch 6/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4183 - accuracy: 0.8313
Epoch 7/50
214/214 [==============================] - 0s 2ms/step - loss: 0.3992 - accuracy: 0.8424
Epoch 8/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4167 - accuracy: 0.8239
Epoch 9/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4182 - accuracy: 0.8295
Epoch 10/50
214/214 [==============================] - 0s 1ms/step - loss: 0.4074 - accuracy: 0.8327

In [30]:
bestclassif = grids.best_estimator_
print(type(bestclassif))
print(grids.best_params_)

<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier'>
{'batch_size': 32, 'epochs': 70, 'optimizer': 'adam'}


In [31]:
y_pred = bestclassif.predict(X_test)
#print(y_pred)
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [32]:
from sklearn.metrics import confusion_matrix
cmat = confusion_matrix(Y_test,y_pred)
print(cmat)

[[1549   46]
 [ 268  137]]
